In [1]:
%cd ..

/Users/nguyenhuyhai/20221/lab_blockchain/orchai_validator


In [2]:
from labeling.upload import ETLProcessor, get_spark, get_batch_intervals, sampling
from omegaconf import OmegaConf

In [3]:
config = OmegaConf.load("config/local_full.yaml")

In [4]:
spark = get_spark()

23/01/18 14:43:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/18 14:43:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
def get_batch_intervals(start_block, end_block, batch_size, vote_proposed_win_size, label_win_size, min_block):
    intervals = []
    while start_block <= end_block:
        batch_start = max(min_block, start_block - vote_proposed_win_size + 1)
        batch_end = batch_start + batch_size - 1
        
        if batch_end > end_block:
            batch_size = end_block - batch_start + 1
            batch_size = (batch_size - vote_proposed_win_size + 1) // combine_win_size * combine_win_size + vote_proposed_win_size - 1
            batch_end = batch_start + batch_size - 1
            
            if batch_start > batch_end:
                break
            
            if batch_size < vote_proposed_win_size + combine_win_size + label_win_size - 1:
                break

        intervals.append((batch_start, batch_end))
        start_block = batch_end - label_win_size + 1
    
    return intervals


vote_proposed_win_size = config.hp.etl.vote_proposed_win_size
label_win_size = config.hp.etl.label_win_size
batch_size = config.hp.upload.batch_size

for bs, be in get_batch_intervals(
    start_block=7063372,
    end_block=7065370,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    label_win_size=label_win_size,
    min_block=7049472
):
    print(bs, be)

In [ ]:
7064871 - 7062873 + 1

In [5]:
df = sampling(spark, config, 7062873, 7064871)

Successfully queried data from database


In [ ]:
df.groupBy("block_height").count().count()

In [6]:
dff = ETLProcessor.data_scoring(df, **config.hp.etl)

------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [ ]:
dff.printSchema()

In [ ]:
dff.groupBy("block_height").count().count()

In [ ]:
dp = dff.groupBy("block_height").count().pandas_api()

In [ ]:
d = dp.values

In [ ]:
bh = d[:, 0]

In [ ]:
bh.sort()

In [ ]:
import numpy as np
from collections import Counter

cnt = Counter(np.diff(bh))
cnt

In [ ]:
bh[0], bh[-1]

In [ ]:
7063372, 7064792
7062873, 7064871

In [ ]:
7063372 - 7062873 + 1

In [ ]:
7064871 - 7064792 + 1